In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import classification_report

In [2]:
# Load data
# ======================
df = pd.read_csv("GYM.csv")

X = df[["Gender", "Goal", "BMI Category"]]
y = df[["Exercise Schedule", "Meal Plan"]]

In [3]:
df

,Gender,Goal,BMI Category,Exercise Schedule,Meal Plan
0,Female,muscle_gain,Normal weight,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
1,Male,fat_burn,Underweight,"Light weightlifting, Yoga, and 2000 steps walking","High-calorie, protein-rich diet: Whole milk, p..."
2,Male,muscle_gain,Normal weight,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
3,Male,muscle_gain,Overweight,"High-intensity interval training (HIIT), Cardi...","Low-carb, high-fiber diet: Avocado, grilled fi..."
4,Female,muscle_gain,Normal weight,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
...,...,...,...,...,...
79995,Male,fat_burn,Normal weight,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
79996,Female,fat_burn,Underweight,"Light weightlifting, Yoga, and 2000 steps walking","High-calorie, protein-rich diet: Whole milk, p..."
79997,Female,muscle_gain,Obesity,"Low-impact cardio, Swimming, and 10000 steps w...","Low-calorie, nutrient-dense diet with portion ..."
79998,Male,fat_burn,Normal weight,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...


In [4]:
X

,Gender,Goal,BMI Category
0,Female,muscle_gain,Normal weight
1,Male,fat_burn,Underweight
2,Male,muscle_gain,Normal weight
3,Male,muscle_gain,Overweight
4,Female,muscle_gain,Normal weight
...,...,...,...
79995,Male,fat_burn,Normal weight
79996,Female,fat_burn,Underweight
79997,Female,muscle_gain,Obesity
79998,Male,fat_burn,Normal weight


In [5]:
y

,Exercise Schedule,Meal Plan
0,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
1,"Light weightlifting, Yoga, and 2000 steps walking","High-calorie, protein-rich diet: Whole milk, p..."
2,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
3,"High-intensity interval training (HIIT), Cardi...","Low-carb, high-fiber diet: Avocado, grilled fi..."
4,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
...,...,...
79995,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...
79996,"Light weightlifting, Yoga, and 2000 steps walking","High-calorie, protein-rich diet: Whole milk, p..."
79997,"Low-impact cardio, Swimming, and 10000 steps w...","Low-calorie, nutrient-dense diet with portion ..."
79998,"Moderate cardio, Strength training, and 5000 s...",Balanced diet with moderate protein and carboh...


In [6]:
# Train / test split FIRST (no leakage)
# ======================
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [7]:
# Encode labels
# ======================
exercise_encoder = LabelEncoder()
meal_encoder = LabelEncoder()

y_ex_train = exercise_encoder.fit_transform(y_train_raw["Exercise Schedule"])
y_ex_test  = exercise_encoder.transform(y_test_raw["Exercise Schedule"])

y_me_train = meal_encoder.fit_transform(y_train_raw["Meal Plan"])
y_me_test  = meal_encoder.transform(y_test_raw["Meal Plan"])

In [8]:
# Encode features
# ======================
ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

X_train = ohe.fit_transform(X_train_raw)
X_test  = ohe.transform(X_test_raw)

In [9]:
# Convert to TensorFlow tensors
# ======================
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test  = tf.convert_to_tensor(X_test, dtype=tf.float32)

y_ex_train = tf.convert_to_tensor(y_ex_train, dtype=tf.int32)
y_ex_test  = tf.convert_to_tensor(y_ex_test, dtype=tf.int32)

y_me_train = tf.convert_to_tensor(y_me_train, dtype=tf.int32)
y_me_test  = tf.convert_to_tensor(y_me_test, dtype=tf.int32)

In [ ]:
# Model definition (shared trunk + two heads)
# ======================
input_dim = X_train.shape[1]
n_exercise = len(np.unique(y_ex_train.numpy()))
n_meal = len(np.unique(y_me_train.numpy()))

inputs = keras.Input(shape=(input_dim,), name="features")

shared = layers.Dense(
    128,
    activation="relu",
    name="shared_dense_1"
)(inputs)

shared = layers.Dense(
    64,
    activation="relu",
    name="shared_dense_2"
)(shared)

exercise_output = layers.Dense(
    n_exercise, activation=None, name="exercise"
)(shared)

meal_output = layers.Dense(
    n_meal, activation=None, name="meal"
)(shared)

model = keras.Model(
    inputs=inputs,
    outputs={
        "exercise": exercise_output,
        "meal": meal_output
    },
    name="multi_output_fitness_model"
)

model.summary()

Model: "multi_output_fitness_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ features            │ (None, 8)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_dense_1      │ (None, 128)       │      1,152 │ features[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_dense_2      │ (None, 64)        │      8,256 │ shared_dense_1[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ exercise (Dense)    │ (None, 4)         │        260 │ shared_dense_2[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ meal (Dense)        │ (None, 4)         │        260 │ shared_dense_2[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,928 (38.78 KB)

 Trainable params: 9,928 (38.78 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Compile
# ======================
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    loss={
        "exercise": keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        "meal": keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    },
    loss_weights={
        "exercise": 1.0,
        "meal": 2.0,   # loss_ex + 2 * loss_me
    }
)

In [12]:
# Training
# ======================
model.fit(
    X_train,
    {
        "exercise": y_ex_train,
        "meal": y_me_train
    },
    epochs=50,
    batch_size=len(X_train),
    verbose=2
)

Epoch 1/50
1/1 - 1s - 983ms/step - exercise_loss: 1.3843 - loss: 4.0682 - meal_loss: 1.3419
Epoch 2/50
1/1 - 0s - 69ms/step - exercise_loss: 1.2800 - loss: 3.4527 - meal_loss: 1.0863
Epoch 3/50
1/1 - 0s - 65ms/step - exercise_loss: 1.1475 - loss: 2.8773 - meal_loss: 0.8649
Epoch 4/50
1/1 - 0s - 71ms/step - exercise_loss: 0.9839 - loss: 2.2590 - meal_loss: 0.6375
Epoch 5/50
1/1 - 0s - 75ms/step - exercise_loss: 0.7912 - loss: 1.6407 - meal_loss: 0.4247
Epoch 6/50
1/1 - 0s - 72ms/step - exercise_loss: 0.5777 - loss: 1.0754 - meal_loss: 0.2488
Epoch 7/50
1/1 - 0s - 70ms/step - exercise_loss: 0.3754 - loss: 0.6289 - meal_loss: 0.1267
Epoch 8/50
1/1 - 0s - 66ms/step - exercise_loss: 0.2106 - loss: 0.3257 - meal_loss: 0.0576
Epoch 9/50
1/1 - 0s - 67ms/step - exercise_loss: 0.1027 - loss: 0.1512 - meal_loss: 0.0243
Epoch 10/50
1/1 - 0s - 72ms/step - exercise_loss: 0.0448 - loss: 0.0642 - meal_loss: 0.0097
Epoch 11/50
1/1 - 0s - 78ms/step - exercise_loss: 0.0183 - loss: 0.0259 - meal_loss: 0.0

In [14]:
# Evaluation
# ======================
preds = model.predict(X_test)

ex_logits = preds["exercise"]
meal_logits = preds["meal"]

ex_preds = np.argmax(ex_logits, axis=1)
meal_preds = np.argmax(meal_logits, axis=1)


print("Exercise Schedule:")
print(classification_report(y_ex_test.numpy(), ex_preds))

print("Meal Plan:")
print(classification_report(y_me_test.numpy(), meal_preds))


500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 591us/step
Exercise Schedule:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3993
           1       1.00      1.00      1.00      4219
           2       1.00      1.00      1.00      3838
           3       1.00      1.00      1.00      3950

    accuracy                           1.00     16000
   macro avg       1.00      1.00      1.00     16000
weighted avg       1.00      1.00      1.00     16000

Meal Plan:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3950
           1       1.00      1.00      1.00      4219
           2       1.00      1.00      1.00      3838
           3       1.00      1.00      1.00      3993

    accuracy                           1.00     16000
   macro avg       1.00      1.00      1.00     16000
weighted avg       1.00      1.00      1.00     16000

